In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


!pip install -q -U tensorflow>=2
import tensorflow as tf



try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import numpy as np
import os
import time


TensorFlow is already loaded. Please restart the runtime to change versions.


Read in the text

In [0]:
corpus = open("corpus.txt", "rb").read().decode(encoding='utf-8')

In [3]:
print ('Length of text: {} characters'.format(len(corpus)))

Length of text: 2782185 characters


In [4]:
print(corpus[:250])

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <!-- The above 3 meta tags *must* come first i


In [5]:
# The unique characters in the file
vocab = sorted(set(corpus))
print ('{} unique characters'.format(len(vocab)))

108 unique characters


Vectorize text- map all unique characters to an integer

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in corpus])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(corpus[:25]), text_as_int[:25]))

'<!DOCTYPE html>\n<html lan' ---- characters mapped to int ---- > [30  3 38 49 37 54 59 50 39  2 73 85 78 77 32  1 30 73 85 78 77  2 77 66
 79]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 50
examples_per_epoch = len(corpus)//(seq_length+1)

In [10]:
#convert stream of text into character indices
# Create training examples / targets

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

<
!
D
O
C


Divide characters into batch size of specific lenght

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta'
' charset="utf-8">\n    <meta http-equiv="X-UA-Compat'
'ible" content="IE=edge">\n    <meta name="viewport" '
'content="width=device-width, initial-scale=1">\n    '
'<!-- The above 3 meta tags *must* come first in the'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <met'
Target data: '!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta'


The input of the RNN will start from '<' and predict '!' in the next time step till the final predicted output is 't' the last character in the sequence before it moves to the next sequence.

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 30 ('<')
  expected output: 3 ('!')
Step    1
  input: 3 ('!')
  expected output: 38 ('D')
Step    2
  input: 38 ('D')
  expected output: 49 ('O')
Step    3
  input: 49 ('O')
  expected output: 37 ('C')
Step    4
  input: 37 ('C')
  expected output: 54 ('T')


In [15]:
# Batch size
BATCH_SIZE = 107

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((107, 50), (107, 50)), types: (tf.int64, tf.int64)>

In [0]:
#Batch size is usually the size of your vocabulary because you will want to maintain a reasonable proportional embedding distance between your vocab



# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(107, 50, 108) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (107, None, 256)          27648     
_________________________________________________________________
gru (GRU)                    (107, None, 1024)         3938304   
_________________________________________________________________
dense (Dense)                (107, None, 108)          110700    
Total params: 4,076,652
Trainable params: 4,076,652
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([ 34,  92, 100,  91,  59,  85,   5,  13,  66, 106,  84, 104,  24,
        20,  17,  11,  87,  77,  12,  69,  95,  40, 102,  49,  62, 101,
        56,   9,  21,  68,  60,  83,  77,  56,   1,  20,  17,   2,  18,
        95,  39,  68,  58,  24,  55,  54,  95,  63,  21,  79])

Checking the sample text generated by this untrained model

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'e comment, delete the comment or reply to the comm'

Next Char Predictions: 
 "@{’zYt#+a└s─62/)vl*d¥F€O\\…V'3cZrlV\n2/ 0¥EcX6UT¥]3n"


Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (107, 50, 108)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.683313


In [0]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
509/509 [==============================] - 1930s 4s/step - loss: 1.5939
Epoch 2/10
509/509 [==============================] - 1942s 4s/step - loss: 0.7561
Epoch 3/10
509/509 [==============================] - 1897s 4s/step - loss: 0.5748
Epoch 4/10
509/509 [==============================] - 1987s 4s/step - loss: 0.5004
Epoch 5/10
509/509 [==============================] - 1966s 4s/step - loss: 0.4588
Epoch 6/10
509/509 [==============================] - 1802s 4s/step - loss: 0.4381
Epoch 7/10
509/509 [==============================] - 1889s 4s/step - loss: 0.4142
Epoch 8/10
509/509 [==============================] - 1862s 4s/step - loss: 0.3976
Epoch 9/10
509/509 [==============================] - 1897s 4s/step - loss: 0.3862
Epoch 10/10
509/509 [==============================] - 1932s 4s/step - loss: 0.3768


In [45]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [47]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 256)            27648     
_________________________________________________________________
gru_7 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_7 (Dense)              (1, None, 108)            110700    
Total params: 4,076,652
Trainable params: 4,076,652
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.5

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [53]:
print(generate_text(model, start_string=u"<!DOCTYPE html> "))

<!DOCTYPE html> <html lang="en">

<head>
  <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
    
                                                                                                    <h3><i class="m-icon-big-swapright m-icon-white"></i> Pages</a>
                            </div>
                                                                                                                              <li><a href="#"><i class="icon-picture"></i> All category</a></li>
                      </ul>
                                      <h2 class="widget-title">Instagram Feed</h2>
                                                                      </div>
                                                                                                                                                                       